# Salary Predictions Based on Job Descriptions

# Part 1 - DEFINE

### ---- 1 Define the problem ----

Requirement is to predict the salaries based on job postings

In [71]:
#import your libraries
import pandas as pd
import sklearn as sk
#etc

#your info here
__author__ = "Sai Lalitha R"
__email__ = "sailalitha.r@gmail.com"

## Part 2 - DISCOVER

### ---- 2 Load the data ----

In [77]:
#load the data into a Pandas dataframe
filepath_train_data="/Users/sailalithasadhu/Desktop/DSDJ/Projects/SalaryPredictions/salaryprediction/data/train_features.csv"
filepath_train_salaries="/Users/sailalithasadhu/Desktop/DSDJ/Projects/SalaryPredictions/salaryprediction/data/train_salaries.csv"
filepath_test_data="/Users/sailalithasadhu/Desktop/DSDJ/Projects/SalaryPredictions/salaryprediction/data/test_features.csv"
train_data = pd.read_csv(filepath_train_data)
train_salaries = pd.read_csv(filepath_train_salaries)
test_data = pd.read_csv(filepath_test_data)
#print(train_data.head())
#train_salaries.head()
print(train_data.shape,train_salaries.shape,test_data.shape)
print(train_data.columns)

(1000000, 8) (1000000, 2) (1000000, 8)
Index(['jobId', 'companyId', 'jobType', 'degree', 'major', 'industry',
       'yearsExperience', 'milesFromMetropolis'],
      dtype='object')


### ---- 3 Clean the data ----

In [80]:
#look for duplicate data, invalid data (e.g. salaries <=0), or corrupt data and remove it

#checking datatypes of columns (o/p: all are correct)
train_data.dtypes
train_salaries.dtypes

#checking for duplicate data (o/p: no duplicates)
train_data.duplicated().sum()
train_salaries.duplicated().sum()

#checking for NaN values (o/p: no NaN values)
train_data.isnull().sum()
train_salaries.isnull().sum()

#checking for invalid data (salary <=0 and removing the row)
invalid_salaries = train_salaries[train_salaries['salary'] <= 0].index
train_salaries.drop(invalid_salaries,inplace=True)
train_data.drop(invalid_salaries,inplace=True)

print(train_data.shape,train_salaries.shape)


(999995, 8) (999995, 2)


### ---- 4 Explore the data (EDA) ----

In [3]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

### ---- 5 Establish a baseline ----

In [5]:
#select a reasonable metric (MSE in this case)
#create an extremely simple model and measure its efficacy
#e.g. use "average salary" for each industry as your model and then measure MSE
#during 5-fold cross-validation

### ---- 6 Hypothesize solution ----

In [ ]:
#brainstorm 3 models that you think may improve results over the baseline model based
#on your 

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc

## Part 3 - DEVELOP

You will cycle through creating features, tuning models, and training/validing models (steps 7-9) until you've reached your efficacy goal

#### Your metric will be MSE and your goal is:
 - <360 for entry-level data science roles
 - <320 for senior data science roles

### ---- 7 Engineer features  ----

In [ ]:
#make sure that data is ready for modeling
#create any new features needed to potentially enhance model

### ---- 8 Create models ----

In [15]:
#create and tune the models that you brainstormed during part 2

### ---- 9 Test models ----

In [1]:
#do 5-fold cross validation on models and measure MSE

### ---- 10 Select best model  ----

In [ ]:
#select the model with the lowest error as your "prodcuction" model

## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

### ---- 12 Deploy solution ----

In [16]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data